# Volume 3: Discrete Hidden Markov Models
    <Name>
    <Class>
    <Date>

In [66]:
import numpy as np
import copy
import string
import codecs

In [75]:
class HMM(object):
    """
    Finite state space hidden markov model.
    """
    
    # Problem 1
    def __init__(self):
        """
        Initialize an HMM with parameters A, B, and pi, initialized as 
        None objects.
        """
        self.pi = None
        self.A = None
        self.B = None
    
    
    # Problem 2
    def _forward(self, z):
        """
        Compute the scaled forward probability matrix and scaling factors.
        
        Parameters
        ----------
        z : ndarray of shape (T,) 
            The sequence of observations
        
        Returns
        -------
        alpha : ndarray of shape (T, n)
            The scaled forward probability matrix
        
        c : ndarray of shape (T,)
            The scaling factors c = [c_0, c_1, ..., c_{T-1}]
        """
        #get initial values
        T = np.shape(z)[0]
        a = np.ones((T, len(self.pi)))
        c = np.ones(T)
        
        #get the first values for alpha and c
        a[0] = self.pi*self.B[z[0]]
        c[0] = 1/(np.sum(a[0]))
        a[0] = c[0]*a[0]
        
        #loop through 1 trhough T to change alpha and c
        for t in range(1, T):
            a[t] = np.sum(a[t-1]*self.A, axis = 1)*self.B[z[t]]
            c[t] = 1/np.sum(a[t])
            a[t] = c[t]*a[t]
            
        return np.array(a),np.array(c)
    
    
    # Problem 3
    def _backward(self, z, c):
        """
        Compute the scaled backward probability matrix.
        
        Parameters
        ----------
        z : ndarray of shape (T,) 
            The sequence of observations
        c : ndarray of shape (T,)
            The scaling factors from the forward pass

        Returns
        -------
        beta : ndarray of shape (T, n)
            The scaled backward probability matrix
        """
        #get initial values
        T = np.shape(z)[0]
        n = len(self.pi)
        B = np.ones((T, n))
        
        #get the first values for alpha and c
        B[-1] = c[-1]*np.ones(n)
        
        #loop through 1 trhough T to change alpha and c
        for t in range(T-2, -1,-1):
            B[t] = np.sum(self.A.T*B[t+1], axis = 1)*self.B[z[t+1]]
            B[t] = np.array([np.sum([self.A[i,j]*B[t+1,i]*self.B[z[t+1],i] for i in range(n)]) for j in range(n)])
            B[t] = c[t]*B[t]
                    
        return np.array(B)
    
    
    # Problem 4
    def _xi(self, z, alpha, beta, c):
        """
        Compute the xi and gamma probabilities.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence
        alpha : ndarray of shape (T, n)
            The scaled forward probability matrix from the forward pass
        beta : ndarray of shape (T, n)
            The scaled backward probability matrix from the backward pass
        c : ndarray of shape (T,)
            The scaling factors from the forward pass

        Returns
        -------
        xi : ndarray of shape (T-1, n, n)
            The xi probability array
        gamma : ndarray of shape (T, n)
            The gamma probability array
        """
        #getting the initial variables
        T = len(z)
        n = len(self.pi)
        xi = np.ones((T-1,n,n))
        r = np.ones((T,n))
        
        #get the xis
        for t in range(T-1):
            for i in range(n):
                for j in range(n):
                    xi[t,i,j] = alpha[t,i]*A[j,i]*self.B[z[t+1],j]*beta[t+1,j]
        #get the rs
        for t in range(T):
            for i in range(n):
                r[t,i] = alpha[t,i]*beta[t,i]/c[t]
                
        return np.array(xi), np.array(r)
    
    # Problem 5
    def _estimate(self, z, xi, gamma):
        """
        Estimate better parameter values and update self.pi, self.A, and
        self.B in place.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence
        xi : ndarray of shape (T-1, n, n)
            The xi probability array
        gamma : ndarray of shape (T, n)
            The gamma probability array
        """
        #getting the initial variables
        T = len(z)
        I, J = self.B.shape
        #editing pi
        self.pi = gamma[0]
        
        #editing A
        self.A = (np.sum(xi, axis = 0)/np.sum(gamma[:-1], axis = 0).reshape(-1,1)).T
        
        #editing B
        nom = lambda i, j: np.sum([gamma[t,j] if z[t]==i else 0 for t in range(T)])
        den = lambda j: np.sum(gamma[:,j])
        self.B = np.array([[nom(i,j)/den(j) for j in range(J)] for i in range(I)])
    
    
    # Problem 6
    def fit(self, z, pi, A, B, max_iter=100, tol=1e-5):
        """
        Fit the model parameters to a given observation sequence.

        Parameters
        ----------
        z : ndarray of shape (T,)
            Observation sequence on which to train the model.
        pi : Stochastic ndarray of shape (n,)
            Initial state distribution
        A : Stochastic ndarray of shape (n, n)
            Initial state transition matrix
        B : Stochastic ndarray of shape (m, n)
            Initial state observation matrix
        max_iter : int
            The maximum number of iterations to take
        tol : float
            The convergence threshold for change in log-probability
        """
        # initialize self.pi, self.A, self.B
        self.pi = pi
        self.A = A
        self.B = B
        
        #Compute log the probabilty
        alpha, c = self._forward(z)
        oldP = -np.inf
        
        # run the iteration
        for i in range(max_iter):
            #Run forward pass
            alpha,c = self._forward(z)
            #Run backward pass
            beta = self._backward(z,c)
            #Compute xi and r probabilties
            xi, r = self._xi(z, alpha, beta,c)
            #Update model parameters
            self._estimate(z, xi, r)
            #Compute the log according to new parameters
            newP = -np.sum(np.log(c))
            
            #if Change in log probabilities is less than e then
            if np.abs(oldP-newP) < tol:
                break
            else:
                oldP = newP

In [76]:
# toy HMM example to be used to check answers
pi = np.array([.6, .4])
A = np.array([[.7, .4],[.3, .6]])
B = np.array([[.1,.7],[.4, .2],[.5, .1]])
z = np.array([0, 1, 0, 2])

h = HMM()
h.pi = pi
h.A = A
h.B = B
alpha, c = h._forward(z)
# print(-np.log(c).sum())                   # problem 2
# -4.6429135909

beta = h._backward(z, c)
# print(beta)                               # problem 3
# [[ 3.1361635   2.89939354]
#  [ 2.86699344  4.39229044]
#  [ 3.898812    2.66760821]
#  [ 3.56816483  3.56816483]]

xi, gamma = h._xi(z, alpha, beta, c)        # problem 4
# print(xi)
# [[[ 0.14166321  0.0465066 ]
#   [ 0.37776855  0.43406164]]
#  [[ 0.17015868  0.34927307]
#   [ 0.05871895  0.4218493 ]]
#  [[ 0.21080834  0.01806929]
#   [ 0.59317106  0.17795132]]]
# print(gamma)
# [[ 0.18816981  0.81183019]
#  [ 0.51943175  0.48056825]
#  [ 0.22887763  0.77112237]
#  [ 0.8039794   0.1960206 ]]

h._estimate(z, xi, gamma)                   # problem 5
# print(h.pi)
# [ 0.18816981  0.81183019]
print(h.A)
# [[ 0.55807991  0.49898142]
#  [ 0.44192009  0.50101858]]
print(h.B)
# [[ 0.23961928  0.70056364]
#  [ 0.29844534  0.21268397]
#  [ 0.46193538  0.08675238]]

[[0.55807991 0.49898142]
 [0.44192009 0.50101858]]
[[0.23961928 0.70056364]
 [0.29844534 0.21268397]
 [0.46193538 0.08675238]]


In [77]:
def vec_translate(a, my_dict):
    # translate numpy array from symbols to state numbers or vice versa
    return np.vectorize(my_dict.__getitem__)(a)

def prep_data(filename):
    # Get the data as a single string
    with codecs.open(filename, encoding='utf-8') as f:
        data=f.read().lower()  # and convert to all lower case

    # remove punctuation and newlines
    remove_punct_map = {ord(char): 
                        None for char in string.punctuation+"\n\r"}
    data = data.translate(remove_punct_map)

    # make a list of the symbols in the data
    symbols = sorted(list(set(data)))

    # convert the data to a NumPy array of symbols
    a = np.array(list(data))

    # make a conversion dictionary from symbols to state numbers
    symbols_to_obsstates = {x:i for i,x in enumerate(symbols)}

    # convert the symbols in a to state numbers
    obs_sequence = vec_translate(a,symbols_to_obsstates)

    return symbols, obs_sequence

## Problem 7

Train a HMM on `declaration.txt`. Use `N=2` states and `M=len(set(obs))=27` observation values ($26$ lower case characters and $1$ whitespace character), and run for $150$ iterations with the deffault value for `tol`.

Once the learning algorithm converges, analyze the state observation matrix $B$. Note which rows correspond to the largest and smallest probability values in each column of $B$, and check the corresponding characters. The HMM should have detected a vowel state and a consonant state.

In [70]:
#getting the information
symbols, obs = prep_data('declaration.txt')

#using hmm
N = 2
M = len(set(obs))
hmm = HMM()

#get the values
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

#use fit
hmm.fit(obs, pi, A, B, max_iter = 150)

In [71]:
#print values
for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}"
          .format(symbols[i], hmm.B[i,0], hmm.B[i,1]))

 , 0.1797, 0.1565
a, 0.0590, 0.0621
b, 0.0329, 0.0000
c, 0.0139, 0.0292
d, 0.0000, 0.0511
e, 0.0703, 0.1328
f, 0.0623, 0.0000
g, 0.0000, 0.0263
h, 0.0000, 0.0707
i, 0.0000, 0.0910
j, 0.0055, 0.0000
k, 0.0000, 0.0028
l, 0.0789, 0.0000
m, 0.0333, 0.0097
n, 0.0000, 0.0979
o, 0.1410, 0.0214
p, 0.0477, 0.0000
q, 0.0021, 0.0000
r, 0.1264, 0.0121
s, 0.0389, 0.0741
t, 0.0032, 0.1278
u, 0.0723, 0.0000
v, 0.0015, 0.0141
w, 0.0028, 0.0180
x, 0.0002, 0.0017
y, 0.0280, 0.0000
z, 0.0000, 0.0008


## Problem 8

Repeat the previous calculation with $3$ hidden states and again with $4$ hidden states. Interpret/explain your results.

In [72]:
#get the info
symbols, obs = prep_data('declaration.txt')

#initialize the info
N = 3
M = len(set(obs))
hmm = HMM()

#get pi and A and B
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

#use fit
hmm.fit(obs, pi, A, B, max_iter = 150)

#print output
for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}"
          .format(symbols[i], hmm.B[i,0], hmm.B[i,1], hmm.B[i,2]))

 , 0.0000, 0.0000, 0.8678
a, 0.0000, 0.1335, 0.0004
b, 0.0000, 0.0266, 0.0000
c, 0.0000, 0.0431, 0.0202
d, 0.0879, 0.0024, 0.0000
e, 0.2899, 0.0158, 0.0000
f, 0.0390, 0.0202, 0.0000
g, 0.0319, 0.0116, 0.0000
h, 0.0342, 0.0713, 0.0000
i, 0.0000, 0.0890, 0.0883
j, 0.0000, 0.0045, 0.0000
k, 0.0010, 0.0031, 0.0000
l, 0.0218, 0.0470, 0.0000
m, 0.0254, 0.0206, 0.0000
n, 0.1176, 0.0441, 0.0000
o, 0.0340, 0.1174, 0.0000
p, 0.0000, 0.0349, 0.0091
q, 0.0000, 0.0005, 0.0029
r, 0.1006, 0.0410, 0.0000
s, 0.1323, 0.0313, 0.0000
t, 0.0518, 0.1391, 0.0000
u, 0.0000, 0.0564, 0.0051
v, 0.0004, 0.0202, 0.0005
w, 0.0026, 0.0252, 0.0000
x, 0.0003, 0.0000, 0.0055
y, 0.0293, 0.0000, 0.0000
z, 0.0000, 0.0011, 0.0000


In [73]:
#get the info
symbols, obs = prep_data('declaration.txt')

#initialize the info
N = 4
M = len(set(obs))
hmm = HMM()

#get pi A and B
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

#use fit
hmm.fit(obs, pi, A, B, max_iter = 150)

# ouptut
for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}, {4:0.4f}"
          .format(symbols[i], hmm.B[i,0], hmm.B[i,1], hmm.B[i,2], hmm.B[i,3]))

 , 0.0000, 0.0000, 0.0000, 0.8915
a, 0.1233, 0.0000, 0.0684, 0.0003
b, 0.0309, 0.0000, 0.0000, 0.0000
c, 0.0598, 0.0000, 0.0000, 0.0000
d, 0.0135, 0.1118, 0.0000, 0.0000
e, 0.0359, 0.2103, 0.2488, 0.0003
f, 0.0232, 0.0577, 0.0000, 0.0000
g, 0.0157, 0.0435, 0.0000, 0.0000
h, 0.0523, 0.0969, 0.0039, 0.0000
i, 0.0635, 0.0000, 0.0686, 0.1079
j, 0.0052, 0.0000, 0.0000, 0.0000
k, 0.0025, 0.0034, 0.0000, 0.0000
l, 0.0326, 0.0061, 0.0819, 0.0000
m, 0.0198, 0.0000, 0.0586, 0.0000
n, 0.0961, 0.0995, 0.0001, 0.0000
o, 0.0965, 0.0446, 0.0931, 0.0000
p, 0.0420, 0.0000, 0.0061, 0.0000
q, 0.0006, 0.0000, 0.0030, 0.0000
r, 0.0281, 0.0061, 0.2308, 0.0000
s, 0.0539, 0.1598, 0.0080, 0.0000
t, 0.1395, 0.1119, 0.0000, 0.0000
u, 0.0210, 0.0000, 0.1019, 0.0000
v, 0.0147, 0.0000, 0.0203, 0.0000
w, 0.0282, 0.0053, 0.0000, 0.0000
x, 0.0000, 0.0000, 0.0064, 0.0000
y, 0.0000, 0.0430, 0.0000, 0.0000
z, 0.0013, 0.0000, 0.0000, 0.0000


## Problem 9

Repeat the same calculation with `WarAndPeace.txt` for $2$ and $3$ hidden states. Interpret/explain your results. Which Cyrillic characters appear to be vowels?

In [78]:
#get the info
symbols, obs = prep_data('WarAndPeace.txt')

#get the initiualized info
N = 2
M = len(set(obs))
hmm = HMM()

#get the pi A and B
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

#use fit
hmm.fit(obs, pi, A, B, max_iter = 150)

#print the output
for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}"
          .format(symbols[i], hmm.B[i,0], hmm.B[i,1]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
 , 0.1647, 0.1637
а, 0.1716, 0.0455
б, 0.0025, 0.0181
в, 0.0301, 0.0416
г, 0.0148, 0.0185
д, 0.0105, 0.0263
е, 0.0083, 0.0823
ж, 0.0033, 0.0096
з, 0.0564, 0.0051
и, 0.0592, 0.0521
й, 0.0002, 0.0111
к, 0.0763, 0.0190
л, 0.0585, 0.0394
м, 0.0116, 0.0257
н, 0.0803, 0.0531
о, 0.0267, 0.1132
п, 0.0090, 0.0268
р, 0.0259, 0.0383
с, 0.0493, 0.0402
т, 0.0151, 0.0546
у, 0.0183, 0.0249
ф, 0.0056, 0.0001
х, 0.0012, 0.0080
ц, 0.0041, 0.0027
ч, 0.0048, 0.0132
ш, 0.0026, 0.0075
щ, 0.0009, 0.0033
ъ, 0.0000, 0.0004
ы, 0.0

In [79]:
#get the info
symbols, obs = prep_data('WarAndPeace.txt')

#get the initizliaed info
N = 3
M = len(set(obs))
hmm = HMM()

#get the pi A and B
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

#use fit
hmm.fit(obs, pi, A, B, max_iter = 150)

#print output
for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}"
          .format(symbols[i], hmm.B[i,0], hmm.B[i,1], hmm.B[i,2]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
 , 0.0000, 0.0065, 0.4937
а, 0.1964, 0.0000, 0.0000
б, 0.0000, 0.0406, 0.0070
в, 0.0000, 0.0729, 0.0501
г, 0.0000, 0.0439, 0.0121
д, 0.0000, 0.0564, 0.0166
е, 0.1840, 0.0000, 0.0059
ж, 0.0000, 0.0229, 0.0037
з, 0.0000, 0.0270, 0.0204
и, 0.1495, 0.0000, 0.0000
й, 0.0000, 0.0000, 0.0274
к, 0.0000, 0.0626, 0.0324
л, 0.0000, 0.0880, 0.0474
м, 0.0000, 0.0369, 0.0345
н, 0.0000, 0.1486, 0.0359
о, 0.2687, 0.0000, 0.0000
п, 0.0000, 0.0656, 0.0081
р, 0.0000, 0.1022, 0.0115
с, 0.0000, 0.0341, 0.0954
т, 0.0000, 0.117